# RecordLinkage Linking

> Leveraging the RL library to determine approximate matching over a range of fields using various methods.

In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

In [ ]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

In [ ]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
dfs = []
for key in sourceFiles.keys():
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])
    print (key)

sports247
rivals
allConf
allAmerican
nflData
ncaa


In [ ]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)
fx.createNewID(idConfig['allConf'], allConf, field_agg)
fx.createNewID(idConfig['allAmerican'], allAmerican, field_agg)
fx.createNewID(idConfig['nflData'], nflData, field_agg)
fx.createNewID(idConfig['ncaa'], ncaa, field_agg, ifAlternate=True)

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
#df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)
df_ncaa = pandas.DataFrame(ncaa)

In [ ]:
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'

In [ ]:
features = pandas.read_pickle("features.pkl")

In [ ]:
features['sum'] = (features['playerName'] + features['college'] + features['city'] + features['state'] + features['highSchool'] + features['year'] + features['position'])/7
features.describe()

,ID,playerName,college,city,state,highSchool,position,year,sum
count,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08,1.111147e+08
mean,3.325932e-04,1.526350e-01,2.815054e-01,1.328318e-01,9.206913e-02,1.266977e-01,2.889289e-02,8.531899e-01,2.382603e-01
std,1.823411e-02,9.488154e-02,3.255962e-01,1.167102e-01,2.891235e-01,9.654791e-02,1.675055e-01,3.539165e-01,6.015086e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.428571e-01
25%,0.000000e+00,8.333333e-02,1.000000e-01,7.142857e-02,0.000000e+00,6.666667e-02,0.000000e+00,1.000000e+00,2.017625e-01
50%,0.000000e+00,1.538462e-01,1.538462e-01,1.111111e-01,0.000000e+00,1.200000e-01,0.000000e+00,1.000000e+00,2.225552e-01
75%,0.000000e+00,2.142857e-01,2.727273e-01,1.875000e-01,0.000000e+00,1.764706e-01,0.000000e+00,1.000000e+00,2.504502e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
features.boxplot(column=['playerName', 'college', 'city', 'state', 'highSchool', 'position', 'year'])

In [ ]:
not_id = features[features['ID'] != 1]
not_id = (not_id.loc[not_id['sum'] > .945])


In [ ]:
not_id.to_pickle("not_id.pkl")

In [ ]:
not_id = pandas.read_pickle("not_id.pkl")

In [ ]:
not_id.reset_index(inplace=True)

In [ ]:
noIDDict = not_id.to_dict('records')
print(noIDDict)

[{'247_index': 134, 'rivals_index': 594, 'ID': 0, 'playerName': 0.6666666666666667, 'college': 1.0, 'city': 1.0, 'state': 1, 'highSchool': 1.0, 'position': 1, 'year': 1, 'sum': 0.9523809523809524}, {'247_index': 138, 'rivals_index': 186, 'ID': 0, 'playerName': 0.6666666666666667, 'college': 1.0, 'city': 1.0, 'state': 1, 'highSchool': 1.0, 'position': 1, 'year': 1, 'sum': 0.9523809523809524}, {'247_index': 150, 'rivals_index': 998, 'ID': 0, 'playerName': 0.875, 'college': 1.0, 'city': 1.0, 'state': 1, 'highSchool': 1.0, 'position': 1, 'year': 1, 'sum': 0.9821428571428571}, {'247_index': 245, 'rivals_index': 1072, 'ID': 0, 'playerName': 0.8333333333333334, 'college': 1.0, 'city': 1.0, 'state': 1, 'highSchool': 0.782608695652174, 'position': 1, 'year': 1, 'sum': 0.9451345755693582}, {'247_index': 276, 'rivals_index': 3789, 'ID': 0, 'playerName': 0.6153846153846154, 'college': 1.0, 'city': 1.0, 'state': 1, 'highSchool': 1.0, 'position': 1, 'year': 1, 'sum': 0.945054945054945}, {'247_index'

In [ ]:
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'
df_sports247.reset_index(inplace=True)
#df_sports247.Index.rename('index', '247_index')
sportDict = df_sports247.to_dict('records')
j=0
for i in sportDict:
    if (j == 0):
        print(i)
    else:
        break
    j = j + 1

{'247_index': 0, 'school': 'nebraska', 'year': '2012', 'playerName': 'paulthurston', 'highSchool': 'Arvada West', 'city': 'Arvada', 'state': 'CO', 'position': 'OT', 'height': 77.0, 'weight': '275', 'compRating': '0.9407', 'compStars': 4, 'nationalRank': '152', 'positionRank': '16', 'stateRank': '2', '247Rating': ' 92  ', '247Stars': 8.0, '247positionRank': '26', '247stateRank': '2', 'displayName': 'Paul Thurston', 'ID': 'paulthurston_nebraska', '247nationalRank': nan}


In [ ]:
## Hey Sober Andrew - this all works now, but you have to clean-up the output
## cause there are dueling column names between the two collections.  
## Look at the summarize code to figure out how to handle that

#Can only include the reset_index the first time
df_sports247.index.name = '247_index'
df_rivals.index.name = 'rivals_index'
df_sports247.reset_index(inplace=True)
df_rivals.reset_index(inplace=True)

sportDict = df_sports247.to_dict('records')
rivalsDict = df_rivals.to_dict('records')

finalSet = []
#print(df_sports247.to_dict('records'))
for record in noIDDict:
    finalRecord = {}
    for recruit in sportDict:
        #print(recruit)
        if (recruit['247_index'] == record['247_index']):
            for key,value in recruit.items():
                finalRecord[key] = value
            break
    for recruit in rivalsDict:
        #print(recruit)
        if (recruit['rivals_index'] == record['rivals_index']):
            for key,value in recruit.items():
                finalRecord[key] = value
            break
    finalSet.append(finalRecord)
print(finalSet)

[{'247_index': 134, 'school': 'rutgers', 'year': '2004', 'playerName': 'joesalinardi', 'highSchool': 'Hudson Catholic', 'city': 'Jersey City', 'state': 'NJ', 'position': 'DT', 'height': 76.0, 'weight': 285.0, 'compRating': '0.7667', 'compStars': 2, 'nationalRank': nan, 'positionRank': nan, 'stateRank': nan, '247Rating': '  N/A  ', '247Stars': 2.0, '247positionRank': 'N/A', '247stateRank': 'N/A', 'displayName': 'Joe Salinardi', 'ID': 'joesalinardi_rutgers', '247nationalRank': nan, 'rivals_index': 594, 'stars': 2, 'rawSchool': nan}, {'247_index': 138, 'school': 'rutgers', 'year': '2004', 'playerName': 'darrensalinardi', 'highSchool': 'Hudson Catholic', 'city': 'Jersey City', 'state': 'NJ', 'position': 'DT', 'height': 76.0, 'weight': 285.0, 'compRating': '0.7667', 'compStars': 2, 'nationalRank': nan, 'positionRank': nan, 'stateRank': nan, '247Rating': '  N/A  ', '247Stars': 2.0, '247positionRank': 'N/A', '247stateRank': 'N/A', 'displayName': 'Darren Salinardi', 'ID': 'darrensalinardi_rutg

In [ ]:
#df_sports247.head()
df_sports247.iloc[5618, :]

In [ ]:
test = df_rivals.iloc[2673, :]
test['year']

In [ ]:
features.iloc[2673, :]

In [ ]:
print(fuzzy_matches)

In [ ]:
ids_to_match = []
ids = fuzzy_matches['ID']
for key in ids.keys():
    i = {}
    print(key[0])
    i['247'] = key[0]
    i['rivals'] = key[1]
    ids_to_match.append(i)


In [ ]:
fuzzy_list = []
for i in ids_to_match:
    sports247_dict = pandas.DataFrame(df_sports247.iloc[i['247'], :]).to_dict()
    
    rivals_dict= pandas.DataFrame(df_rivals.iloc[i['rivals'], :]).to_dict()
    for i in rivals_dict:   
        sports247_dict.update(i)
    fuzzy_list.append(sport247_dict)

In [ ]:
print(fuzzy_list)

In [ ]:
df_rivals